### Imports en setup

In [1]:
import torch
import torch.nn as nn
from loguru import logger

### Device check

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

logger.info(f"Device: {device}")

2025-12-29 16:30:42.505 | INFO     | __main__:<module>:8 - Device: cuda


## Stap 1: Simpele Conv laag begrijpen

Een Conv2d laag transformeert een image naar feature maps

In [3]:
# Maak een simpele conv laag
# in_channels=3 (RGB), out_channels=16 (filters), kernel=3x3
simple_conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)

# Test met dummy input (batch=1, channels=3, height=128, width=128)
dummy_input = torch.randn(1, 3, 128, 128)
output = simple_conv(dummy_input)

logger.info(f"Input shape:  {dummy_input.shape}")
logger.info(f"Output shape: {output.shape}")

2025-12-29 16:31:58.227 | INFO     | __main__:<module>:9 - Input shape:  torch.Size([1, 3, 128, 128])
2025-12-29 16:31:58.232 | INFO     | __main__:<module>:10 - Output shape: torch.Size([1, 16, 128, 128])


## Stap 2: Conv blok met alle componenten

Een typisch conv blok: Conv -> BatchNorm -> ReLU -> Dropout -> MaxPool

In [4]:
conv_block = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Houdt zelfde grootte
    nn.BatchNorm2d(32),                          # Normaliseert activaties
    nn.ReLU(),                                   # Activatie functie
    nn.Dropout2d(0.2),                           # Regularisatie
    nn.MaxPool2d(kernel_size=2),                 # Halveert width/height
)

output = conv_block(dummy_input)
logger.info(f"Input:  {dummy_input.shape}")
logger.info(f"Output: {output.shape}")  # Moet (1, 32, 64, 64) zijn

2025-12-29 16:33:03.282 | INFO     | __main__:<module>:10 - Input:  torch.Size([1, 3, 128, 128])
2025-12-29 16:33:03.285 | INFO     | __main__:<module>:11 - Output: torch.Size([1, 32, 64, 64])


## Stap 3: Meerdere conv blokken stapelen
Elke MaxPool halveert de afmetingen

In [5]:
def make_conv_block(in_channels, out_channels, dropout=0.2, use_batchnorm=True):
    """Maakt één conv blok met optionele batchnorm."""
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)]
    
    if use_batchnorm:
        layers.append(nn.BatchNorm2d(out_channels))
    
    layers.append(nn.ReLU())
    layers.append(nn.Dropout2d(dropout))
    layers.append(nn.MaxPool2d(kernel_size=2))
    
    return nn.Sequential(*layers)

In [6]:
# Test met 3 conv blokken
block1 = make_conv_block(3, 32)   # 128 -> 64
block2 = make_conv_block(32, 32)  # 64 -> 32
block3 = make_conv_block(32, 32)  # 32 -> 16

x = dummy_input
logger.info(f"Start: {x.shape}")

x = block1(x)
logger.info(f"Na blok 1: {x.shape}")

x = block2(x)
logger.info(f"Na blok 2: {x.shape}")

x = block3(x)
logger.info(f"Na blok 3: {x.shape}")

2025-12-29 16:35:27.737 | INFO     | __main__:<module>:7 - Start: torch.Size([1, 3, 128, 128])
2025-12-29 16:35:27.749 | INFO     | __main__:<module>:10 - Na blok 1: torch.Size([1, 32, 64, 64])
2025-12-29 16:35:27.750 | INFO     | __main__:<module>:13 - Na blok 2: torch.Size([1, 32, 32, 32])
2025-12-29 16:35:27.757 | INFO     | __main__:<module>:16 - Na blok 3: torch.Size([1, 32, 16, 16])


## Stap 4: Van feature maps naar classificatie

Flatten de feature maps en gebruik Linear lagen

In [7]:
# Na 3 conv blokken: (batch, 32, 16, 16)
# Flatten: 32 * 16 * 16 = 8192 features
flatten_size = 32 * 16 * 16

dense_layers = nn.Sequential(
    nn.Flatten(),
    nn.Linear(flatten_size, 128),  # hidden_units
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 5),  # 5 flower classes
)

# Test de hele keten
x = dummy_input
x = block1(x)
x = block2(x)
x = block3(x)
output = dense_layers(x)

logger.info(f"Final output: {output.shape}")  # Moet (1, 5) zijn

2025-12-29 16:37:09.959 | INFO     | __main__:<module>:20 - Final output: torch.Size([1, 5])


## Stap 5: Flatten size automatisch berekenen

Het probleem: bij andere num_conv_layers verandert de flatten size

In [8]:
def calculate_conv_output(conv_layers, input_shape):
    """Berekent output size door test forward pass."""
    dummy = torch.zeros(1, *input_shape)
    with torch.no_grad():
        output = conv_layers(dummy)
    return output.view(1, -1).size(1)

# Test met verschillende aantal lagen
input_shape = (3, 128, 128)

# 2 lagen
conv_2_layers = nn.Sequential(
    make_conv_block(3, 32),
    make_conv_block(32, 32),
)
size_2 = calculate_conv_output(conv_2_layers, input_shape)
logger.info(f"2 conv lagen -> flatten size: {size_2}")

# 4 lagen
conv_4_layers = nn.Sequential(
    make_conv_block(3, 32),
    make_conv_block(32, 32),
    make_conv_block(32, 32),
    make_conv_block(32, 32),
)
size_4 = calculate_conv_output(conv_4_layers, input_shape)
logger.info(f"4 conv lagen -> flatten size: {size_4}")

2025-12-29 16:38:34.288 | INFO     | __main__:<module>:17 - 2 conv lagen -> flatten size: 32768
2025-12-29 16:38:34.304 | INFO     | __main__:<module>:27 - 4 conv lagen -> flatten size: 2048


## Stap 6: Het configureerbare model

Alles samenvoegen in een class met config dict

In [9]:
class ConfigurableCNN(nn.Module):
    """CNN met instelbare hyperparameters."""
    
    def __init__(self, config: dict):
        super().__init__()
        
        # Hyperparameters uit config
        self.num_conv_layers = config.get("num_conv_layers", 3)
        self.num_filters = config.get("num_filters", 32)
        self.dropout = config.get("dropout", 0.2)
        self.hidden_units = config.get("hidden_units", 128)
        self.use_batchnorm = config.get("use_batchnorm", True)
        
        # Vaste parameters
        self.input_channels = config.get("input_channels", 3)
        self.num_classes = config.get("num_classes", 5)
        self.img_size = config.get("img_size", 128)
        
        # Bouw conv lagen
        self.convolutions = self._build_conv_layers()
        
        # Bereken flatten size
        conv_output = self._calculate_conv_output()
        logger.info(f"Conv output: {conv_output} features")
        
        # Bouw dense lagen
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_output, self.hidden_units),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_units, self.num_classes),
        )
    
    def _build_conv_layers(self):
        """Bouwt conv lagen dynamisch."""
        layers = []
        in_ch = self.input_channels
        
        for _ in range(self.num_conv_layers):
            layers.append(nn.Conv2d(in_ch, self.num_filters, kernel_size=3, padding=1))
            if self.use_batchnorm:
                layers.append(nn.BatchNorm2d(self.num_filters))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout2d(self.dropout))
            layers.append(nn.MaxPool2d(kernel_size=2))
            in_ch = self.num_filters
        
        return nn.Sequential(*layers)
    
    def _calculate_conv_output(self):
        """Berekent flatten size met dummy input."""
        dummy = torch.zeros(1, self.input_channels, self.img_size, self.img_size)
        with torch.no_grad():
            out = self.convolutions(dummy)
        return out.view(1, -1).size(1)
    
    def forward(self, x):
        x = self.convolutions(x)
        x = self.dense(x)
        return x

## Stap 7: Test verschillende configuraties

In [10]:
# Config 1: Klein model
config_small = {
    "num_conv_layers": 2,
    "num_filters": 16,
    "dropout": 0.1,
    "hidden_units": 64,
    "use_batchnorm": False,
}

model_small = ConfigurableCNN(config_small)
params_small = sum(p.numel() for p in model_small.parameters())
logger.info(f"Klein model: {params_small:,} parameters")

2025-12-29 16:43:05.142 | INFO     | __main__:__init__:24 - Conv output: 16384 features
2025-12-29 16:43:05.147 | INFO     | __main__:<module>:12 - Klein model: 1,051,733 parameters


In [11]:
# Config 2: Groot model
config_large = {
    "num_conv_layers": 4,
    "num_filters": 64,
    "dropout": 0.3,
    "hidden_units": 256,
    "use_batchnorm": True,
}

model_large = ConfigurableCNN(config_large)
params_large = sum(p.numel() for p in model_large.parameters())
logger.info(f"Groot model: {params_large:,} parameters")

2025-12-29 16:43:40.512 | INFO     | __main__:__init__:24 - Conv output: 4096 features
2025-12-29 16:43:40.524 | INFO     | __main__:<module>:12 - Groot model: 1,163,205 parameters


In [12]:
# Test forward pass
test_input = torch.randn(32, 3, 128, 128)
output = model_large(test_input)
logger.info(f"Output shape: {output.shape}")  # Moet (32, 5) zijn

2025-12-29 16:44:16.041 | INFO     | __main__:<module>:4 - Output shape: torch.Size([32, 5])


## Stap 8: Combineer met echte data

In [ ]:
# Gebruik de data uit eerdere notebook cellen
X_aug, y_aug = next(train.stream())
logger.info(f"Input shape: {X_aug.shape}")

# Forward pass
model = ConfigurableCNN({"num_conv_layers": 3, "num_filters": 32})
model = model.to(device)

output = model(X_aug.to(device))
logger.info(f"Predictions shape: {output.shape}")
logger.info(f"Predictions sample: {output[0]}")